<a href="https://colab.research.google.com/github/beloveddie/AI-Craft/blob/main/LLM_Response_Validation_Workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q llama-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.6/263.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 7.6 MB/s eta 0:00:00


In [ ]:
# get OPENAI_API_KEY_SOLID and set as OPENAI_API_KEY env variable
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_KEY_SOLID')

In [ ]:
from llama_index.core.workflow import (
    Event,
    StartEvent,
    StopEvent,
    Workflow,
    step
)
from llama_index.llms.openai import OpenAI
from typing import List, Dict, Any


# Output Validation Workflow - Customer Support Focus
class ResponseEvent(Event):
    query: str
    response: str

class ValidatedResponseEvent(Event):
    response: str
    validation_result: Dict[str, Any]

class ResponseValidationFlow(Workflow):
    llm = OpenAI(model="gpt-4.1-mini")

    @step
    async def generate_response(self, ev: StartEvent) -> ResponseEvent:
        query = ev.query

        prompt = f"""
        Act as a customer support assistant and respond to the following query:

        QUERY: "{query}"

        Provide a helpful, friendly response that addresses the query directly.
        """

        response = await self.llm.acomplete(prompt)
        return ResponseEvent(query=query, response=str(response))

    @step
    async def validate_response(self, ev: ResponseEvent) -> ValidatedResponseEvent:
        response = ev.response

        prompt = f"""
        Validate the following customer support response based on our brand guidelines:

        RESPONSE: "{response}"

        Analyze the response for:
        1. Tone alignment (friendly, professional, empathetic)
        2. Brand voice consistency
        3. Solution completeness
        4. Policy compliance
        5. Proper disclaimers where needed
        6. Absence of incorrect/misleading information

        Return a JSON object with:
        {{
            "compliance_score": float (0-1),
            "tone_score": float (0-1),
            "completeness_score": float (0-1),
            "issues": [specific issues found],
            "suggestions": [improvement recommendations]
        }}

        Without the JSON formatting syntax.
        """

        validation_result = await self.llm.acomplete(prompt)
        return ValidatedResponseEvent(
            response=response,
            validation_result=dict(validation_result)
        )

    @step
    async def finalize_response(self, ev: ValidatedResponseEvent) -> StopEvent:
        validation_result = ev.validation_result

        # In a real implementation, we would parse the validation result
        # and potentially modify the response or reject it if it doesn't meet standards
        prompt = f"""
        Based on this validation result:
        {validation_result}

        Determine if this response meets our quality standards (compliance_score > 0.8).
        If not, revise the response to address the issues while maintaining the core message.

        Return a JSON object with:
        {{
            "revised_response": str,
            "complaince_score": float (0-1),
            "completeness_score": float (0-1),
        }}
        """

        final_response = await self.llm.acomplete(prompt)

        return StopEvent(result={
            "assistant_response": ev.response,
            "final_validation_response": str(final_response)
        })

In [ ]:
# Response Validation Example
import json
response_flow = ResponseValidationFlow(timeout=60, verbose=False)
response_result = await response_flow.run(query="I've been waiting for my refund for 2 weeks now. This is unacceptable!")
print(json.dumps(response_result, indent=2))

{
  "assistant_response": "Hello! I\u2019m really sorry to hear that your refund has been delayed and understand how frustrating this must be. Let me look into your order right away to see what\u2019s causing the delay. Could you please provide me with your order number or any related details? We\u2019ll do our best to resolve this for you as quickly as possible. Thank you for your patience!",
  "final_validation_response": "{\n  \"revised_response\": \"Thank you for reaching out regarding your refund. Typically, our refund processing takes 5-7 business days. For more details, you can visit our refund policy page on our website. To assist you further, could you please provide your order number and the email address used for the purchase? Rest assured, your information will be handled confidentially and used solely for this inquiry. Please note that external factors, such as payment processor delays, may occasionally affect refund timelines. Once I have your details, I will review your 

In [ ]:
type(response_result)

dict

In [ ]:
print(response_result.get('assistant_response'))

Hello! I’m really sorry to hear that your refund has been delayed and understand how frustrating this must be. Let me look into your order right away to see what’s causing the delay. Could you please provide me with your order number or any related details? We’ll do our best to resolve this for you as quickly as possible. Thank you for your patience!


In [ ]:
print(response_result.get('final_validation_response'))

{
  "revised_response": "Thank you for reaching out regarding your refund. Typically, our refund processing takes 5-7 business days. For more details, you can visit our refund policy page on our website. To assist you further, could you please provide your order number and the email address used for the purchase? Rest assured, your information will be handled confidentially and used solely for this inquiry. Please note that external factors, such as payment processor delays, may occasionally affect refund timelines. Once I have your details, I will review your order and get back to you within 24 hours with an update.",
  "complaince_score": 0.95,
  "completeness_score": 0.95
}
